<a href="https://colab.research.google.com/github/zombienok/short_quest_generator/blob/main/quest_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/zombienok/short_quest_generator.git
!cd short_quest_generator
!pip install -r /content/short_quest_generator/requirements.txt
!curl -fsSL https://ollama.com/install.sh | sh

Cloning into 'short_quest_generator'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 23 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 7.11 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
import subprocess
import time
import ollama


# Запускаем сервер Ollama в фоне (если ещё не запущен)
proc = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ждём, пока сервер запустится (обычно 3-5 секунд)
time.sleep(5)

# Проверяем, работает ли сервер
try:
    ollama.pull("owl/t-lite:latest")
    print(ollama.list())  # Должен вывести список моделей (пустой, если ничего не загружено)
except Exception as e:
    print("Ошибка:", e)

models=[Model(model='owl/t-lite:latest', modified_at=datetime.datetime(2025, 8, 3, 21, 19, 38, 890585, tzinfo=TzInfo(UTC)), digest='52bcfdc27e1b605bb7905cfbfaca2cf5541e85f86c1e1138f0e00d11336242b1', size=4661229593, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_0'))]


In [ ]:
import ollama


model_to_use = "owl/t-lite:latest"
generation_config = {
    "temperature": 0,  # Баланс креативности/скорости,
    "seed": 42,
}


messages = [
    {
        "role": "system",
        "content": """
Учитывая сеттинг, ГГ и цель, создай JSON с сюжетом квеста, состоящим из 3 веток, и 3 сценами подряд в каждой (т.е. всего 9+2 сцена (+2 - старт и финал)), где:
1. Каждая сцена имеет уникальный scene_id и next_scene для переходов. ID каждой сцены должен быть уникальным. Каждая сцена имеет text, в котором вкратце описывается сама сцена
2. В каждой не финальной/провальной сцене 3 варианта выбора (choices). choices - список доступных игроку вариантов действий в данной сцене. Каждый элемент массива choices должен содержать 3 поля: text,after,next_scene.
text - текст выбора (например, "Подскользнуться на кожуре от банана", "Станцевать тектоник", "Молча сверлить взглядом"),
after - последствия выбора ("Вы упали и сломали ногу"), next_scene - ID следующей сцены.
Пример choices:
"choices": [
  {
    "text": "Подскользнуться на кожуре от банана",
    "after": "Вы подскользнулись. Чувствуете себя глупо. Но на потолке виден код от сейфа!",
    "next_scene": "opening_safe_scene"
  },
  {
    "text": "Станцевать тектоник",
    "after": "Вы станцевали тектоник. Вы еблан?",
    "next_scene": "are_u_dumb_scene"
  }
]
Условия для генерации вариантов:
-варианты должны быть смешными, но по делу
-Среди генерируемых вариантов могут присутствовать провальные варианты. Они приводят к провалу квеста.
-все варианты ведут либо к следующей существующей сцене, либо к провалу, либо к финалу квеста
Примеры choices:
Пример формата вывода:
```json
{
"scene_id": "start_scene",
"text": "...",
"choices": [
{
"text": "...",
"next_scene": "...",
"after": "..."
},
{
"text": "...",
"next_scene": "...",
"after": "..."
}
]
},
...
{"scene_id": "final_scene",
"text": "...",
"choices": []
}```
В ответе JSON с 9+2 сценами (НЕ МЕНЬШЕ). СТРОГО по формату.
"""
},
    {"role": "user",
    "content": """Сеттинг: S.T.A.L.K.E.R., ГГ: Меченый. спокойный парень с дробовиком, Цель: пробраться в лабораторию X-16 и украсть документы"""}
]


outputs = ollama.chat(
    model = model_to_use,
    messages = messages,
    options = generation_config,)
print(outputs)

KeyboardInterrupt: 

In [ ]:
print("""Придумай общую структуру квеста из 3 веток (A, B, C) с 3 сценами в каждой + стартовая и финальная сцены. Выведи только список scene_id в порядке их следования с кратким описанием связи между ними (1-2 слова). Пример:\n"start_scene" -> "branchA_scene1" (убежать)\n"branchA_scene1" -> "branchA_scene2" (драка)\n...\n"branchC_scene3" -> "final_scene" (победа)""")

In [ ]:
print(outputs["message"]["content"])

```json
{
  "scene_id": "start_scene",
  "text": "Вы оказались на окраине Зоны Отчуждения. Вокруг вас шуршат кустарники, а из-за деревьев доносится жуткий гул. Вы стоите перед входом в заброшенный дом.",
  "choices": [
    {
      "text": "Проверить подвал дома",
      "next_scene": "entering_house",
      "after": "Входя внутрь, вы замечаете следы недавнего пребывания. В подвале обнаруживаете старый рюкзак с аптечкой и картой."
    },
    {
      "text": "Идти напрямую к лаборатории",
      "next_scene": "direct_to_lab",
      "after": "Вы быстро продвигаетесь через лес, но вдруг слышите шум. Ваши инстинкты подсказывают вам остановиться."
    },
    {
      "text": "Проверить ближайший бункер на карте",
      "next_scene": "bunker_check",
      "after": "Вы находите заброшенный военный бункер, но внутри нет ничего полезного. Время тикает..."
    }
  ]
}

{
  "scene_id": "entering_house",
  "text": "В подвале вы нашли старый рюкзак и карту.",
  "choices": [
    {
      "text": "Осмотре

In [4]:
prompt_choices = """
Опиши варианты, последствия, описания сцен детальнее, сохраняя их количество.
Дополни варианты для сцен строго так:
branchA_scene1:
Смешной: Действие -> {ПОСЛЕДСТВИЕ} + branchA_scene2
Провальный: Действие -> failure_why
Логичный: Действие -> {ПОСЛЕДСТВИЕ} + branchA_scene3
Никаких символов *, -, "". Только текст по шаблону.
Ожидаемый формат (варианты могут быть другими):
-branchA_scene1:
  Смешной: "Станцевать тектоник" → {"Бармен присоединяется, теряет бдительность"} + branchA_scene2
  Провальный: "Выпить неизвестную жидкость" → "Отравление, game over"
  Логичный: "Подкупить охранника" → "branchA_scene3"
"""



In [6]:
# import ollama
# import json



MODEL_TO_USE = "owl/t-lite:latest"
extracted_info = {
    "setting": "Киберпанк",
    "main_charachter": "Хакер-одиночка",
    "quest": "взломать замок на двери, найти и забрать чип с вирусом"
}

messages_strict = [{
    "role": "system",
    "content": "Ты генератор структур для квестов. Отвечаешь ТОЛЬКО в указанном формате. Никаких комментариев, пояснений, символов *, -, >."
}]

prompt_structure = f"""
Сеттинг: {extracted_info["setting"]}
ГГ: {extracted_info["main_charachter"]}
Цель: {extracted_info["quest"]}

Выведи 8 сцен в строгом формате:
1. start_scene: Действие1 -> scene_id1, Действие2 -> scene_id2, Действие3 -> scene_id3
2. scene_id1: Действие1 -> scene_id4, Действие2 -> fail_причина, Действие3 -> scene_id5
...
8. final_scene:

Правила:
- Всего столько строк, сколько создано сцен
- В каждой строке ровно 1,2 или 3 варианта.
- Только действия и ID сцен/провалов
- Никаких других символов или текста
"""
print(prompt_structure)
# response_structure = ollama.chat(
#     model=MODEL_TO_USE,
#     messages=messages_strict + [{"role": "user", "content": prompt_structure}]
# )
# structure = response_structure["message"]["content"]  # Чистая структура
# print(structure)


Сеттинг: Киберпанк
ГГ: Хакер-одиночка
Цель: взломать замок на двери, найти и забрать чип с вирусом

Выведи 8 сцен в строгом формате:
1. start_scene: Действие1 -> scene_id1, Действие2 -> scene_id2, Действие3 -> scene_id3
2. scene_id1: Действие1 -> scene_id4, Действие2 -> fail_причина, Действие3 -> scene_id5
...
8. final_scene: 

Правила:
- Всего 8 строк (start + 6 сцен + final)
- В каждой строке ровно 3 варианта
- Только действия и ID сцен/провалов
- Никаких других символов или текста



In [6]:
prompt_choices = """
Опиши варианты, последствия, описания сцен детальнее, сохраняя их количество.
Дополни варианты для сцен строго так:
branchA_scene1:
Смешной: Действие -> {ПОСЛЕДСТВИЕ} + branchA_scene2
Провальный: Действие -> failure_why
Логичный: Действие -> {ПОСЛЕДСТВИЕ} + branchA_scene3
Никаких символов *, -, "". Только текст по шаблону.
Ожидаемый формат (варианты могут быть другими):
-branchA_scene1:
  Смешной: "Станцевать тектоник" → {"Бармен присоединяется, теряет бдительность"} + branchA_scene2
  Провальный: "Выпить неизвестную жидкость" → "Отравление, game over"
  Логичный: "Подкупить охранника" → "branchA_scene3"
"""
response_choices = ollama.chat(
    model=MODEL_TO_USE,
    options = {"temperature": 0.5},
    messages=messages_strict + [{"role": "user", "content": structure}]
)
choices = response_choices["message"]["content"]  # Чистые варианты
print(choices)

Отлично! Это действительно интересный формат квеста и его развитие. Такой подход позволяет создать сложную многоходовую историю с множеством вариантов исходов, что добавляет глубину и увлекательность игровому процессу.

### Описание квеста

#### Начальная сцена (start_scene)
1. **Взлом двери**:
   - Если взять отмычку: переход в `branchA_scene1`
   - Если установить взрывчатка: переход в `branchB_scene1`
   - Если забыть об отмычке или сделать что-то другое: сцена `finalA_scene2` (провал миссии)

#### Взлом двери с помощью отмычки (branchA_scene1)
1. **Замереть и ждать**:
   - Если замереть долго, переход в `branchA_scene2`
   
2. **Использовать отмычку**:
   - Успех: переход в `finalA_scene` (дверь открыта), успех взлома.
   - Неудачный ход: переход в `failure_stuck`

#### Действия после взламывания двери (branchA_scene2)
1. **Забыть отмычку**:
   - Переход в `finalA_scene2` с неудачей миссии
   
2. **Остановиться и использовать сканер**:
   - Если обнаружить что-то ценное: переход в 

In [9]:
json_strict = """
Преврати ПОЛНОСТЬЮ квест в JSON. Строго соблюдай:
- имеющийся сеттинг, ГГ и цель
- изначальное количество сцен
- У всех сцен: scene_id, text, choices (text, after, next_scene).
- Провалы ведут на fail_сцены.
Выведи ТОЛЬКО JSON, начиная с '{' и заканчивая '}'.
Никакого текста до/после!
Ожидаемый формат сцены (варианты другие):
{
"scene_id": "village_gate",
"text": "Вы подходите к воротам деревни. Страж в броне пристально смотрит на
вас.",
"choices": [
{
"text": "Засосаться со стражем",
"after": "Вы неожиданно наклоняетесь вперёд и делаете суровому стражу французский поцелуй",
"next_scene": "kissing_guard"
},
{
"text": "Пойти обратно в лес",
"after": "Вы разворачиваетесь и подскальзываетесь на банановой кожуре. У вас сломана нога",
"next_scene": "broken_leg"
}
]
}
"""
messages_strict_json = [{
        "role": "system",
        "content": json_strict}]
response_json = ollama.chat(
    model=MODEL_TO_USE,
    options = {"temperature": 0.5},
    messages=messages_strict_json + [{"role": "user", "content": structure}],
    format = "json"
)

print(response_json["message"]["content"])

{
  "start_scene": {
    "scene_id": "start_scene",
    "text": "Вы подходите к двери охраняемого объекта.",
    "choices": [
      {
        "text": "Взломать",
        "after": "Начинается взлом двери. Вы подбираете отмычку и начинаете работать.",
        "next_scene": "branchA_scene1"
      },
      {
        "text": "Замереть и ждать",
        "after": "Вы медленно приближаетесь к объекту, но не заметили ничего подозрительного. Дверь закрыта.",
        "next_scene": "finalA_scene2"
      }
    ]
  },
  
  "branchA_scene1": {
    "scene_id": "branchA_scene1",
    "text": "Отмычка работает, и дверной замок щелкнул. Вы открываете её.",
    "choices": [
      {
        "text": "Использовать отмычку",
        "after": "Вы осторожно проникаете внутрь объекта.",
        "next_scene": "finalA_scene"
      },
      {
        "text": "Забыть о взломе и уйти",
        "after": "Ваше внимание отвлекли, и вы решаете не продолжать. Дверь захлопнулась за вами.",
        "next_scene": "failure_loc

In [ ]:
# Извлечение JSON (на случай, если модель "добавила мусор")
quest_json = json.loads(
    re.search(r'\{.*\}', response_json["response"], flags=re.DOTALL).group(0)
)

print(quest_json)  # Готовый JSON для работы